In [1]:
import tensorflow as tf
import tensorflow_recommenders as tfrs
import numpy as np
import pandas as pd
from tensorflow.keras.callbacks import EarlyStopping
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
import json
from collections import Counter

In [2]:
# recipe_df = pd.read_csv("recipes_data.csv", nrows=1000)
# recipe_df.to_csv("recipes_1000.csv", index=False)

recipe_df = pd.read_csv("recipes_1000.csv")

In [3]:
recipe_df

,title,ingredients,directions,link,source,NER,site
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""bite size shredded rice biscuits"", ""vanilla""...",www.cookbooks.com
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""cream of mushroom soup"", ""beef"", ""sour cream...",www.cookbooks.com
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""pepper"", ""cream cheese"", ""gar...",www.cookbooks.com
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken gravy"", ""cream of mushroom soup"", ""c...",www.cookbooks.com
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""graham cracker crumbs"", ""powdered sugar"", ""p...",www.cookbooks.com
...,...,...,...,...,...,...,...
995,Heath Bar Pie,"[""3 Heath bars, chopped fine"", ""1 medium conta...","[""Mix chopped Heath bars with whipped topping ...",www.cookbooks.com/Recipe-Details.aspx?id=976718,Gathered,"[""chocolate curls"", ""graham cracker pie crust""]",www.cookbooks.com
996,Victorian Baked French Toast,"[""1 c. brown sugar"", ""1/3 c. butter"", ""2 Tbsp....","[""Cook brown sugar, butter and corn syrup in s...",www.cookbooks.com/Recipe-Details.aspx?id=908190,Gathered,"[""bread"", ""light corn syrup"", ""brown sugar"", ""...",www.cookbooks.com
997,Quick Swedish Meatballs,"[""1 lb. ground beef"", ""1 c. soft bread crumbs""...","[""Combine meat, bread crumbs, cheese, soup mix...",www.cookbooks.com/Recipe-Details.aspx?id=850050,Gathered,"[""onion soup"", ""nutmeg"", ""cream cheese"", ""stro...",www.cookbooks.com
998,Irish Stew(Microwave),"[""2 lb. lamb, cut in 1-inch cubes"", ""2 c. wate...","[""In 4-quart casserole, combine lamb, 1 1/4 cu...",www.cookbooks.com/Recipe-Details.aspx?id=1017368,Gathered,"[""potatoes"", ""onion soup"", ""lamb"", ""water"", ""f...",www.cookbooks.com


In [4]:
recipe_df = recipe_df.drop(columns=["source", "site"])
recipe_df

,title,ingredients,directions,link,NER
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,"[""bite size shredded rice biscuits"", ""vanilla""..."
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,"[""cream of mushroom soup"", ""beef"", ""sour cream..."
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,"[""frozen corn"", ""pepper"", ""cream cheese"", ""gar..."
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,"[""chicken gravy"", ""cream of mushroom soup"", ""c..."
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,"[""graham cracker crumbs"", ""powdered sugar"", ""p..."
...,...,...,...,...,...
995,Heath Bar Pie,"[""3 Heath bars, chopped fine"", ""1 medium conta...","[""Mix chopped Heath bars with whipped topping ...",www.cookbooks.com/Recipe-Details.aspx?id=976718,"[""chocolate curls"", ""graham cracker pie crust""]"
996,Victorian Baked French Toast,"[""1 c. brown sugar"", ""1/3 c. butter"", ""2 Tbsp....","[""Cook brown sugar, butter and corn syrup in s...",www.cookbooks.com/Recipe-Details.aspx?id=908190,"[""bread"", ""light corn syrup"", ""brown sugar"", ""..."
997,Quick Swedish Meatballs,"[""1 lb. ground beef"", ""1 c. soft bread crumbs""...","[""Combine meat, bread crumbs, cheese, soup mix...",www.cookbooks.com/Recipe-Details.aspx?id=850050,"[""onion soup"", ""nutmeg"", ""cream cheese"", ""stro..."
998,Irish Stew(Microwave),"[""2 lb. lamb, cut in 1-inch cubes"", ""2 c. wate...","[""In 4-quart casserole, combine lamb, 1 1/4 cu...",www.cookbooks.com/Recipe-Details.aspx?id=1017368,"[""potatoes"", ""onion soup"", ""lamb"", ""water"", ""f..."


In [5]:
# Fungsi untuk bersihkan dan konversi string list menjadi list Python asli
def clean_and_parse_json_list(x):
    if isinstance(x, str):
        # Ganti double double-quote menjadi satu double-quote
        x = x.replace('""', '"')

        # Kadang ada tanda kutip tunggal yang harus diganti ke double quotes
        # Untuk data kamu, coba replace tanda kutip tunggal di awal dan akhir
        if x.startswith("'") and x.endswith("'"):
            x = x[1:-1]

        # Pastikan tanda kutip didalam string di-escape dengan benar
        try:
            return json.loads(x)
        except json.JSONDecodeError:
            # Kalau gagal, coba raw string fix (misalnya escape manual)
            x = x.replace('\\"', '"').replace('\\\'', '\'')
            try:
                return json.loads(x)
            except Exception as e:
                print("Gagal parsing:", e)
                # fallback return string asli kalau error parsing
                return x
    return x

# Terapkan hanya ke kolom yang berbentuk list dalam string
for col in ["ingredients", "directions", "NER"]:
    recipe_df[col] = recipe_df[col].apply(clean_and_parse_json_list)

# Contoh cek hasil parsing kolom NER baris pertama
print(recipe_df["NER"].iloc[0])

# Kalau mau convert kolom NER jadi string (optional, misal untuk model input string)
recipe_df["NER_str"] = recipe_df["NER"].apply(lambda x: ", ".join(x) if isinstance(x, list) else "")

# Cek hasilnya
print(recipe_df["NER_str"].iloc[0])

Gagal parsing: Expecting ',' delimiter: line 1 column 22 (char 21)
Gagal parsing: Expecting ',' delimiter: line 1 column 81 (char 80)
['bite size shredded rice biscuits', 'vanilla', 'brown sugar', 'nuts', 'milk', 'butter']
bite size shredded rice biscuits, vanilla, brown sugar, nuts, milk, butter


In [6]:
# Buat user dan resep
recipes = recipe_df["title"].unique()
num_users = 5000
user_ids = [f"user_{i+1}" for i in range(num_users)]

# Bangun interactions
interactions = []
np.random.seed(42)

for user in user_ids:
    liked = np.random.choice(recipes, size=50, replace=False)
    for recipe in liked:
        interactions.append({"user_id": user, "title": recipe})

interaction_df = pd.DataFrame(interactions)

# Atur ulang seed untuk rating
np.random.seed(42)

# Jumlah total interaksi
n = len(interaction_df)

# Buat 80% rating > 3 (yaitu: 4 atau 5)
high_ratings = np.random.randint(4, 6, size=int(n * 0.9))  # 4 or 5
# Buat 20% rating <= 3 (1, 2, atau 3)
low_ratings = np.random.randint(1, 4, size=n - len(high_ratings))

# Gabungkan dan acak
all_ratings = np.concatenate([high_ratings, low_ratings])
np.random.shuffle(all_ratings)

# Masukkan ke DataFrame
interaction_df["rating"] = all_ratings

# Tampilkan tabel
interaction_df

,user_id,title,rating
0,user_1,Calico Beans,4
1,user_1,Toll House Cookies,4
2,user_1,Pineapple Parfait Pie,5
3,user_1,Dirt Cups,4
4,user_1,Ground Beef Casserole,4
...,...,...,...
249995,user_5000,Spaghetti Meat Sauce,4
249996,user_5000,St. Joseph'S Zeppole,2
249997,user_5000,Hobo Bread(Brown Bread),4
249998,user_5000,Beer Bread,4


In [9]:
# ====================
# 1. Data Preparation
# ====================
interaction_df["user_id"] = interaction_df["user_id"].astype(str)
interaction_df["title"] = interaction_df["title"].astype(str)

unique_user_ids = interaction_df["user_id"].unique()
unique_titles = interaction_df["title"].unique()

train_df, test_df = train_test_split(interaction_df, test_size=0.2, random_state=42)

train_ds = tf.data.Dataset.from_tensor_slices({
    "user_id": train_df["user_id"].values,
    "title": train_df["title"].values,
    "rating": train_df["rating"].values
}).shuffle(100).batch(768).cache()

val_ds = tf.data.Dataset.from_tensor_slices({
    "user_id": test_df["user_id"].values,
    "title": test_df["title"].values,
    "rating": test_df["rating"].values,
}).batch(512)

recipe_dataset = tf.data.Dataset.from_tensor_slices(unique_titles)

# ====================
# 2. Lookup Layers
# ====================
user_lookup = tf.keras.layers.StringLookup(vocabulary=unique_user_ids, mask_token=None)
title_lookup = tf.keras.layers.StringLookup(vocabulary=unique_titles, mask_token=None)

# ====================
# 3. User & Item Model
# ====================
embedding_dim = 512

class UserModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.user_embedding = tf.keras.Sequential([
            user_lookup,
            tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dim),
            tf.keras.layers.Dense(1024, activation="relu"),
            tf.keras.layers.Dense(embedding_dim)
        ])
    def call(self, inputs):
        return self.user_embedding(inputs)

class ItemModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.item_embedding = tf.keras.Sequential([
            title_lookup,
            tf.keras.layers.Embedding(len(unique_titles) + 1, embedding_dim),
            tf.keras.layers.Dense(1024, activation="relu"),
            tf.keras.layers.Dense(embedding_dim)
        ])
    def call(self, inputs):
        return self.item_embedding(inputs)

# ====================
# 4. Combined Model with Metrics
# ====================
class RecipeModel(tfrs.Model):
    def __init__(self, user_model, item_model):
        super().__init__()
        self.user_model = user_model
        self.item_model = item_model
        self.retrieval_task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
                candidates=recipe_dataset.batch(128).map(lambda x: (x, item_model(x)))
            )
        )
        self.rating_task = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()]
        )

    def compute_loss(self, features, training=False):
        user_embeddings = self.user_model(features["user_id"])
        item_embeddings = self.item_model(features["title"])

        rating_predictions = tf.reduce_sum(user_embeddings * item_embeddings, axis=1)

        rating_loss = self.rating_task(
            labels=features["rating"],
            predictions=rating_predictions
        )
        retrieval_loss = self.retrieval_task(user_embeddings, item_embeddings)

        return rating_loss + retrieval_loss

# ====================
# 5. Training
# ====================
user_model = UserModel()
item_model = ItemModel()
model = RecipeModel(user_model, item_model)

model.compile(optimizer=tf.keras.optimizers.Adam(0.0005))

early_stop = EarlyStopping(
    monitor="val_loss", 
    patience=5, 
    restore_best_weights=True
)

model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=100,
    callbacks=[early_stop]
)

# ====================
# 6. Build Recommendation Index
# ====================
def map_title_to_embedding(title):
    embedding = item_model(tf.expand_dims(title, 0))
    return tf.expand_dims(title, 0), embedding

indexed_dataset = recipe_dataset.map(map_title_to_embedding)

index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(indexed_dataset)


Epoch 1/100
261/261 [==============================] - 78s 292ms/step - factorized_top_k/top_1_categorical_accuracy: 9.6500e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0056 - factorized_top_k/top_10_categorical_accuracy: 0.0111 - factorized_top_k/top_50_categorical_accuracy: 0.0507 - factorized_top_k/top_100_categorical_accuracy: 0.0955 - root_mean_squared_error: 1.6267 - loss: 5081.2155 - regularization_loss: 0.0000e+00 - total_loss: 5081.2155 - val_factorized_top_k/top_1_categorical_accuracy: 3.2000e-04 - val_factorized_top_k/top_5_categorical_accuracy: 0.0035 - val_factorized_top_k/top_10_categorical_accuracy: 0.0084 - val_factorized_top_k/top_50_categorical_accuracy: 0.0490 - val_factorized_top_k/top_100_categorical_accuracy: 0.1022 - val_root_mean_squared_error: 0.9544 - val_loss: 1956.2574 - val_regularization_loss: 0.0000e+00 - val_total_loss: 1956.2574
Epoch 2/100
261/261 [==============================] - 76s 291ms/step - factorized_top_k/top_1_categorical_accuracy: 2

In [10]:
# ====================
# 9. Contoh Penggunaan
# ====================
user_id = "user_3"  # Pastikan ID dalam format string dan cocok dengan user_lookup
user_interactions = interaction_df[interaction_df["user_id"] == user_id]

print(f"Resep yang telah diberi rating oleh user {user_id}:")
for _, row in user_interactions.iterrows():
    print(f"- {row['title']} (Rating: {row['rating']})")

# Set judul yang sudah dikunjungi
visited_titles = set(user_interactions["title"].values)

# Ambil rekomendasi dari model
scores, titles = index(tf.constant([user_id]), k=50)

print(f"\nRekomendasi untuk user {user_id} (resep yang belum di-rate):")
count = 0
max_results = 20

for score, title in zip(scores[0].numpy(), titles[0].numpy()):
    title_str = title.decode("utf-8")
    if title_str in visited_titles:
        continue
    print(f"- {title_str} (Skor: {score:.4f})")
    count += 1
    if count >= max_results:
        break

if count == 0:
    print("Tidak ada rekomendasi resep baru untuk ditampilkan.")
elif count < max_results:
    print(f"(Hanya {count} resep baru yang bisa direkomendasikan)")


Resep yang telah diberi rating oleh user user_3:
- Yellow Cake(Delicious With Just A Dusting Of Powdered Sugar.)   (Rating: 5)
- Sausages And Seashells (Rating: 5)
- Tennessee Tea (Rating: 5)
- Warm Apple Crisp (Rating: 4)
- Baked Corn (Rating: 5)
- Squash Souffle (Rating: 4)
- Frozen Salad (Rating: 5)
- Chicken Maranda (Rating: 5)
- Nutty Oven Fried Chicken (Rating: 4)
- Stir Fried Veggies Medley (Rating: 5)
- Hot Fruit Punch (Rating: 5)
- Cabbage And Noodles (Rating: 4)
- Persimmon Cookies (Rating: 4)
- Quick Fruit Salad (Rating: 4)
- Orange-Coconut Pie (Rating: 4)
- Gran'S Toll House Cookies (Rating: 5)
- Chocolate Macaroon Bars (Rating: 5)
- Wacky Cake (Rating: 4)
- Corn Casserole (Rating: 3)
- Cheeseburger Potato Soup (Rating: 5)
- Life Saver Cookies (Rating: 5)
- Hamburger Potato Hot Dish (Rating: 5)
- Almond Roco (Rating: 5)
- Creole Green Beans (Rating: 3)
- Brownie Pie (Rating: 4)
- Texas Prize Cake (Rating: 4)
- Cheeseburger Crescent Bake(1 Hour)   (Rating: 4)
- Buckeyes(Cook

In [16]:
# ====================
# Data dan variabel awal
# ====================
item_popularity = Counter(interaction_df["title"])
total_interaction = sum(item_popularity.values())
all_items = set(interaction_df["title"].unique())

user_history = interaction_df.groupby("user_id")["title"].apply(set).to_dict()

train_df, test_df = train_test_split(interaction_df, test_size=0.2, random_state=42)

# ====================
# Precision@K
# ====================
def precision_at_k(k):
    correct = 0
    total = 0
    test_dict = test_df.groupby("user_id")["title"].apply(set).to_dict()

    for user_id, true_items in test_dict.items():
        _, recommended = index(tf.constant([user_id]), k=k)
        recommended_ids = set(title.numpy().decode("utf-8") for title in recommended[0])
        hits = recommended_ids.intersection(true_items)
        correct += len(hits)
        total += k
    return correct / total if total > 0 else 0

# ====================
# Novelty@K
# ====================
def novelty_at_k(k):
    total_novelty = 0.0
    total_users = 0

    for user_id in test_df["user_id"].unique():
        _, recommended = index(tf.constant([user_id]), k=k)
        novelty_score = 0.0
        for title in recommended[0].numpy():
            title_str = title.decode("utf-8")
            freq = item_popularity[title_str] / total_interaction
            if freq > 0:
                novelty_score += -np.log2(freq)
        avg_novelty = novelty_score / k
        total_novelty += avg_novelty
        total_users += 1

    return total_novelty / total_users if total_users > 0 else 0

# ====================
# Coverage@K
# ====================
def coverage_at_k(k):
    recommended_items = set()
    for user_id in test_df["user_id"].unique():
        _, recommended = index(tf.constant([user_id]), k=k)
        for title in recommended[0].numpy():
            recommended_items.add(title.decode("utf-8"))
    return len(recommended_items) / len(all_items)

# ============================
# Serendipity Precomputation
# ============================
title_to_embedding = {}
for title in interaction_df["title"].unique():
    vec = model.item_model(tf.constant([title]))
    vec = tf.math.l2_normalize(vec, axis=1)
    title_to_embedding[title] = vec.numpy()[0]

user_mean_embeddings = {}
for user_id in test_df["user_id"].unique():
    seen = user_history.get(user_id, set())
    seen_vecs = [title_to_embedding[title] for title in seen if title in title_to_embedding]
    if not seen_vecs:
        continue
    seen_avg = np.mean(seen_vecs, axis=0)
    seen_avg = seen_avg / np.linalg.norm(seen_avg)
    user_mean_embeddings[user_id] = seen_avg

# ============================
# Serendipity@K
# ============================
def make_serendipity_function(k):
    def compute_serendipity_batch(user_id_tensor):
        user_id = user_id_tensor.numpy().decode("utf-8") if isinstance(user_id_tensor.numpy(), bytes) else str(user_id_tensor.numpy())

        seen = user_history.get(user_id, set())
        if not seen or user_id not in user_mean_embeddings:
            return tf.constant(0.0, dtype=tf.float32)

        seen_avg = user_mean_embeddings[user_id].reshape(1, -1)
        _, recommended = index(tf.constant([user_id]), k=k)
        recommended_titles = recommended[0].numpy()

        ser = 0
        valid_k = 0
        for title in recommended_titles:
            title_str = title.decode("utf-8")
            if title_str in seen or title_str not in title_to_embedding:
                continue
            rec_vec = title_to_embedding[title_str].reshape(1, -1)
            dot_sim = np.dot(seen_avg, rec_vec.T)[0][0]
            ser += 1 - dot_sim
            valid_k += 1

        score = ser / valid_k if valid_k > 0 else 0.0
        return tf.constant(score, dtype=tf.float32)

    return compute_serendipity_batch

def serendipity_at_k_fast(k):
    compute_fn = make_serendipity_function(k)
    user_ids_tensor = tf.constant(list(user_mean_embeddings.keys()), dtype=tf.string)

    serendipity_scores = tf.data.Dataset.from_tensor_slices(user_ids_tensor) \
        .map(lambda uid: tf.py_function(func=compute_fn, inp=[uid], Tout=tf.float32)) \
        .batch(128) \
        .map(lambda batch: tf.reduce_sum(batch)) \
        .reduce(tf.constant(0.0), lambda acc, val: acc + val)

    return serendipity_scores.numpy() / len(user_mean_embeddings)

# ============================
# Evaluasi Semua Metrik
# ============================
def evaluate_model(k_values=[10]):
    print("Evaluasi sistem rekomendasi:")
    for k in k_values:
        print(f"\nTop-{k}")
        precision = precision_at_k(k)
        novelty = novelty_at_k(k)
        coverage = coverage_at_k(k)
        serendipity = serendipity_at_k_fast(k)
        print(f"- Precision@{k}:   {precision:.4f}")
        print(f"- Novelty@{k}:     {novelty:.4f}")
        print(f"- Coverage@{k}:    {coverage:.4f}")
        print(f"- Serendipity@{k}: {serendipity:.4f}")

# ============================
# Jalankan Evaluasi
# ============================
evaluate_model(k_values=[10, 50, 100])

Evaluasi sistem rekomendasi:

Top-10
- Precision@10:   0.0089
- Novelty@10:     9.8406
- Coverage@10:    0.7525
- Serendipity@10: 0.0348

Top-50
- Precision@50:   0.0099
- Novelty@50:     9.8333
- Coverage@50:    0.9516
- Serendipity@50: 0.0341

Top-100
- Precision@100:   0.0103
- Novelty@100:     9.8311
- Coverage@100:    0.9857
- Serendipity@100: 0.0338
